<a href="https://colab.research.google.com/github/tausif04/Data-Mining-and-Machine-Learning-Lab/blob/main/Copy_of_DM%26MLE_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df=pd.read_csv('loan_application_data.csv')
df

,Age,Income,CreditScore,YearsEmployed,Education,Job,Location,LoanApproved
0,37,61895.42,730,2.86,Bachelor,Admin,Urban,1
1,39,60021.77,673,8.41,High School,Technical,Urban,1
2,25,53841.30,757,2.24,High School,Technical,Urban,1
3,45,47780.28,650,7.01,High School,Management,Suburban,1
4,40,53748.86,635,8.48,High School,Technical,Urban,0
...,...,...,...,...,...,...,...,...
93,40,55784.91,701,4.88,High School,Technical,Urban,1
94,38,46736.47,684,2.42,High School,Admin,Urban,1
95,34,50984.40,763,5.19,High School,Technical,Urban,1
96,45,61795.90,684,10.97,High School,Admin,Urban,1


In [ ]:
print("Missing Values Per Column:")
print(df.isnull().sum())

Missing Values Per Column:
Age              0
Income           0
CreditScore      0
YearsEmployed    0
Education        0
Job              0
Location         0
LoanApproved     0
dtype: int64


In [ ]:
#one-hot encoding for Location and Job
onehot_encoded= pd.get_dummies(df, columns=['Location','Job' ],drop_first=True)
print("One_Hot Encoded Column:")
print(onehot_encoded.head())

One_Hot Encoded Column:
   Age    Income  CreditScore  YearsEmployed    Education  LoanApproved  \
0   37  61895.42          730           2.86     Bachelor             1   
1   39  60021.77          673           8.41  High School             1   
2   25  53841.30          757           2.24  High School             1   
3   45  47780.28          650           7.01  High School             1   
4   40  53748.86          635           8.48  High School             0   

   Location_Suburban  Location_Urban  Job_Executive  Job_Management  \
0              False            True          False           False   
1              False            True          False           False   
2              False            True          False           False   
3               True           False          False            True   
4              False            True          False           False   

   Job_Technical  
0          False  
1           True  
2           True  
3          False  
4  

In [ ]:
# Frequency encoding for job
freq_encode = df['Job'].value_counts()
df['Job_Freq_Encoded'] = df['Job'].map(freq_encode)

print("Frequency Mapping:")
print(freq_encode)

Frequency Mapping:
Job
Admin         41
Technical     39
Management    14
Executive      4
Name: count, dtype: int64


In [ ]:
#label encoding for education
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Education_LabelEncoded'] = le.fit_transform(df['Education'])
print("Label Encoded Column:")
print("Original : ",df['Education'].unique())
print("Lebel_Encoded : ",df['Education_LabelEncoded'].unique())
df[['Education','Education_LabelEncoded']].head()


Label Encoded Column:
Original :  ['Bachelor' 'High School' 'Master' 'PhD']
Lebel_Encoded :  [0 1 2 3]


,Education,Education_LabelEncoded
0,Bachelor,0
1,High School,1
2,High School,1
3,High School,1
4,High School,1


In [ ]:
# target encoding for Job
job_target_map = df.groupby('Job')['LoanApproved'].mean().to_dict()
df['Job_Target_Encoded'] = df['Job'].map(job_target_map)
print("Target Encoded Column:")
print(df[['Job', 'Job_Target_Encoded']].head())

Target Encoded Column:
          Job  Job_TargetEncoded
0       Admin           0.853659
1   Technical           0.743590
2   Technical           0.743590
3  Management           0.785714
4   Technical           0.743590


In [ ]:
def det_outliers(df,values):
  # Calculate IQR
  Q1 = df[values].quantile(0.25)
  Q3 = df[values].quantile(0.75)
  IQR = Q3 - Q1

  # Define outlier bounds
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Identify outliers
  outliers = df[(df[values] < lower_bound) | (df[values] > upper_bound)]
  # Remove outliers
  df_cleaned = df[(df[values] >= lower_bound) & (df[values] <= upper_bound)]
  return outliers,df_cleaned

columns= ['Age', 'Income', 'CreditScore', 'YearsEmployed']

for i in columns:
  outliers ,df_cleaned = det_outliers(df,i)
  print(f"Outliers in '{i}':")
  print(outliers[[i]].head())
  print(f"DataFrame after removing '{i}' outliers:")
  print(df_cleaned.head())




Outliers in 'Age':
    Age
50   17
56   58
DataFrame after removing 'Age' outliers:
   Age    Income  CreditScore  YearsEmployed    Education         Job  \
0   37  61895.42          730           2.86     Bachelor       Admin   
1   39  60021.77          673           8.41  High School   Technical   
2   25  53841.30          757           2.24  High School   Technical   
3   45  47780.28          650           7.01  High School  Management   
4   40  53748.86          635           8.48  High School   Technical   

   Location  LoanApproved  
0     Urban             1  
1     Urban             1  
2     Urban             1  
3  Suburban             1  
4     Urban             0  
Outliers in 'Income':
      Income
45  83074.06
66  84879.61
71  84222.75
72  27642.92
DataFrame after removing 'Income' outliers:
   Age    Income  CreditScore  YearsEmployed    Education         Job  \
0   37  61895.42          730           2.86     Bachelor       Admin   
1   39  60021.77          673   

In [ ]:
from scipy.stats import zscore
def det_outlier_Zscore(df,values):
  # Calculate Z-scores
  df_temp=df.copy()
  df_temp['z_score'] = zscore(df_temp[values])

  # Identify outliers
  outliers = df_temp[df_temp['z_score'].abs() > 3]
  # Remove outliers
  df_cleaned = df_temp[df_temp['z_score'].abs() <= 3]
  return outliers,df_cleaned
columns= ['Age', 'Income', 'CreditScore', 'YearsEmployed']

for i in columns:
  outliers , df_cleaned= det_outlier_Zscore(df,i)
  print(f"Outliers in '{i}':")
  print(outliers[[i]].head())
  print(f"DataFrame after removing '{i}' outliers:")
  print(df_cleaned.head())



Outliers in 'Age':
    Age
56   58
DataFrame after removing 'Age' outliers:
   Age    Income  CreditScore  YearsEmployed    Education         Job  \
0   37  61895.42          730           2.86     Bachelor       Admin   
1   39  60021.77          673           8.41  High School   Technical   
2   25  53841.30          757           2.24  High School   Technical   
3   45  47780.28          650           7.01  High School  Management   
4   40  53748.86          635           8.48  High School   Technical   

   Location  LoanApproved   z_score  
0     Urban             1  0.149391  
1     Urban             1  0.425622  
2     Urban             1 -1.508000  
3  Suburban             1  1.254318  
4     Urban             0  0.563738  
Outliers in 'Income':
Empty DataFrame
Columns: [Income]
Index: []
DataFrame after removing 'Income' outliers:
   Age    Income  CreditScore  YearsEmployed    Education         Job  \
0   37  61895.42          730           2.86     Bachelor       Admin   
1